In [9]:
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn

from torch.distributions import Categorical #unlike Normal, it has discrete probability distribution--best for action--rather than regression tasks/uncertainties

import gymnasium as gym

In [12]:
#class Vanilla(nn.module):
 #   def __init__(self, num_obs_dims: int, num_action_dims: int, num_hidden_dims = 64): #128 is a common practice power of two value but many powers of two work well 
  #      super().__init__() #accessing constructor of Vanilla aka nn.module
   #     self.fc1 = nn.Linear(n_obs_dim, n_hidden_dim) #fully connected linear layer 
    #    self.fc2 = nn.Linear(n_hidden_dim, n_action_dim)
    #def forward(self, torch.Tensor) -> torch.Tensor: 